In [2]:
pip install spacy


^C
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [95 lines of output]
  Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
    Using cached setuptools-75.3.0-py3-none-any.whl.metadata (6.9 kB)
    Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
    Using cached cymem-2.0.10.tar.gz (10 kB)
    Installing build dependencies: started
    Installing build dependencies: finished with status 'done'
    Getting requirements to build wheel: started
    Getting requirements to build wheel: finished with status 'done'
    Preparing metadata (pyproject.toml): started
    Preparing metadata (pyproject.toml): finished with status 'done'
    Using cached preshed-3.0.9-cp38-cp38-win_amd64.whl.metadata (2.2 kB)
    Using cached murmurhash-1.0.11.tar.gz (13 kB)
    Installing build dependencies: started
    Installing build dependencies: finished with status 'done'
    Get

  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'


In [3]:
# Import additional libraries for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import spacy
from textblob import TextBlob
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import contractions
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# Set style for visualizations
plt.style.use('seaborn')
sns.set_palette("husl")


In [ ]:
# Load the data
df = pd.read_csv('/kaggle/input/sentiments/task3/train3.csv')
print("Dataset Overview:")
print(f"Number of tweets: {len(df)}")
print(f"\nDataset Info:")
print(df.info())

In [ ]:
import nltk

# Download all required resources explicitly
nltk.download('wordnet')
nltk.download('omw-1.4')  # Open Multilingual WordNet
nltk.download('punkt')
nltk.download('stopwords')

# Then initialize your lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
df = df.dropna()

In [ ]:
nan_count = df.isna().sum()
nan_count


In [ ]:
def simple_preprocess_text(text):
    """
    A simpler preprocessing function that doesn't rely on WordNet
    """
    # Convert to string if not already
    text = str(text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove hashtags and mentions
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Handle contractions
    text = contractions.fix(text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Simple tokenization by splitting on whitespace
    tokens = text.split()
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join tokens back into text
    return ' '.join(tokens)

# Try the simpler preprocessing if needed
df['processed_text'] = df['Text'].apply(simple_preprocess_text)

In [ ]:
df.head

In [ ]:
# 1. Basic Dataset Analysis
print("\n1. Distribution of Sentiments")
sentiment_dist = df['category'].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(x=sentiment_dist.index, y=sentiment_dist.values)
plt.title('Distribution of Sentiments')
plt.xlabel('category')
plt.ylabel('Count')
plt.show()

In [ ]:
# Load the data
dft = pd.read_csv('/kaggle/input/sentiments/task3/test3.csv')
print("Dataset Overview:")
print(f"Number of tweets: {len(df)}")
print(f"\nDataset Info:")
print(df.info())

In [ ]:
df.describe

In [ ]:
# Calculate class imbalance
print("\nClass Distribution Percentages:")
print(df['category'].value_counts(normalize=True) * 100)



In [ ]:
# 2. Text Length Analysis
df['text_length'] = df['Text'].str.len()
df['word_count'] = df['Text'].str.split().str.len()

plt.figure(figsize=(15, 5))

# Text length distribution
plt.subplot(1, 2, 1)
sns.boxplot(x='category', y='text_length', data=df)
plt.title('Text Length by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Text Length')


In [ ]:

# Word count distribution
plt.subplot(1, 2, 2)
sns.boxplot(x='category', y='word_count', data=df)
plt.title('Word Count by Sentiment')
plt.xlabel('Sentiment')
plt.ylabel('Word Count')
plt.tight_layout()
plt.show()

print("\nText Length Statistics:")
print(df.groupby('category')['text_length'].describe())

In [ ]:
# Word Distribution Analysis
def analyze_word_distributions(text_series, category_series):
    """
    Analyzes word usage patterns across different categories to identify distinctive
    language patterns that might help in sentiment classification.
    """
    print("Analyzing word distributions across categories...")
    
    # Create word frequency distributions for each category
    category_word_dist = {}
    for category in category_series.unique():
        # Get texts for this category
        category_texts = text_series[category_series == category]
        
        # Combine all texts and clean
        combined_text = ' '.join(category_texts).lower()
        # Remove URLs, mentions, special characters
        cleaned_text = re.sub(r'http\S+|www\S+|@\w+|#\w+|[^a-zA-Z\s]', '', combined_text)
        # Tokenize and remove stopwords
        words = [word for word in word_tokenize(cleaned_text) 
                if word not in stop_words and len(word) > 2]
        
        # Store word frequencies
        category_word_dist[category] = Counter(words)
        
        # Plot top words for this category
        plt.figure(figsize=(12, 6))
        common_words = dict(Counter(words).most_common(15))
        plt.bar(common_words.keys(), common_words.values())
        plt.title(f'Most Common Words in Category: {category}')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()
        
        print(f"\nTop 15 words in category {category}:")
        for word, count in Counter(words).most_common(15):
            print(f"{word}: {count}")
    
    return category_word_dist

# Perform word distribution analysis
word_distributions = analyze_word_distributions(df['Text'], df['category'])

In [ ]:
# Analyzing Distinctive Words
def analyze_distinctive_words(word_distributions):
    """
    Identifies words that are strongly associated with particular categories,
    which can be valuable features for classification.
    """
    print("\nAnalyzing distinctive words for each category...")
    
    # Calculate word importance scores using TF-IDF-like approach
    total_categories = len(word_distributions)
    distinctive_words = {}
    
    for category, word_dist in word_distributions.items():
        # Calculate distinctiveness score for each word
        word_scores = {}
        for word, count in word_dist.items():
            # Count in how many other categories this word appears
            category_presence = sum(1 for other_dist in word_distributions.values()
                                 if word in other_dist)
            
            # Calculate distinctiveness score (similar to TF-IDF)
            distinctiveness = count * np.log(total_categories / category_presence)
            word_scores[word] = distinctiveness
        
        # Store top distinctive words
        distinctive_words[category] = dict(sorted(word_scores.items(), 
                                                key=lambda x: x[1], 
                                                reverse=True)[:10])
        
        print(f"\nMost distinctive words for category {category}:")
        for word, score in distinctive_words[category].items():
            print(f"{word}: {score:.2f}")

# Analyze distinctive words
analyze_distinctive_words(word_distributions)


In [ ]:
# Bigram Analysis
def analyze_bigrams(text_series, category_series):
    """
    Analyzes common word pairs to understand contextual patterns in language
    usage across categories.
    """
    print("\nAnalyzing bigram patterns...")
    
    for category in category_series.unique():
        # Get texts for this category
        category_texts = text_series[category_series == category]
        
        # Combine and clean texts
        combined_text = ' '.join(category_texts).lower()
        cleaned_text = re.sub(r'http\S+|www\S+|@\w+|#\w+|[^a-zA-Z\s]', '', combined_text)
        
        # Generate bigrams
        tokens = [word for word in word_tokenize(cleaned_text) 
                 if word not in stop_words and len(word) > 2]
        bigram_freq = Counter(ngrams(tokens, 2))
        
        # Plot top bigrams
        plt.figure(figsize=(12, 6))
        common_bigrams = dict(bigram_freq.most_common(10))
        plt.bar([' '.join(bg) for bg in common_bigrams.keys()], 
                common_bigrams.values())
        plt.title(f'Most Common Bigrams in Category: {category}')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()
        
        print(f"\nTop 10 bigrams in category {category}:")
        for bigram, count in bigram_freq.most_common(10):
            print(f"{' '.join(bigram)}: {count}")

# Perform bigram analysis
analyze_bigrams(df['Text'], df['category'])

In [ ]:
# Text Length Analysis
def analyze_text_length_patterns(text_series, category_series):
    """
    Examines how text length varies across categories, which can reveal
    relationships between message length and sentiment.
    """
    print("\nAnalyzing text length patterns...")
    
    # Calculate text lengths
    text_lengths = text_series.str.len()
    word_counts = text_series.str.split().str.len()
    
    # Create length distributions plot
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=category_series, y=text_lengths)
    plt.title('Text Length Distribution by Category')
    plt.xlabel('Category')
    plt.ylabel('Number of Characters')
    plt.show()
    
    # Print statistical summary
    print("\nText length statistics by category:")
    length_stats = pd.DataFrame({
        'text_length': text_lengths,
        'word_count': word_counts,
        'category': category_series
    }).groupby('category').describe()
    print(length_stats)

# Analyze text length patterns
analyze_text_length_patterns(df['Text'], df['category'])

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['processed_text'], 
    df['category'],
    test_size=0.2, 
    random_state=42,
    stratify=df['category']
)


In [ ]:
# Create TF-IDF vectors
print("Creating TF-IDF vectors...")
tfidf = TfidfVectorizer(
    max_features=5000,  # Limit features to prevent overfitting
    ngram_range=(1, 2), # Use both unigrams and bigrams
    min_df=5            # Minimum document frequency
)

# Transform the text data into TF-IDF features
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

def train_evaluate_svm(kernel='linear'):
    print(f"\nTraining SVM with {kernel} kernel...")
    
    # Initialize SVM
    svm_model = SVC(
        kernel=kernel,
        C=1.0,
        random_state=42,
        probability=True
    )
    
    # Train the model
    svm_model.fit(X_train_tfidf, y_train)
    
    # Make predictions
    y_pred = svm_model.predict(X_test_tfidf)
    
    # Print classification report
    print(f"\nClassification Report for {kernel} kernel SVM:")
    print(classification_report(y_test, y_pred))
    
    return svm_model, y_pred

# Train and evaluate both Linear and RBF kernel SVMs
print("\n=== Linear Kernel SVM ===")
linear_svm, linear_predictions = train_evaluate_svm('linear')



In [ ]:
def perform_grid_search(kernel='linear'):
    print(f"\nPerforming Grid Search for {kernel} kernel...")
    
    # Define parameter grid
    param_grid = {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto'] if kernel == 'rbf' else ['scale']
    }
    
    # Initialize SVM
    svm = SVC(kernel=kernel, random_state=42)
    
    # Perform grid search
    grid_search = GridSearchCV(
        svm,
        param_grid,
        cv=3,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit grid search
    grid_search.fit(X_train_tfidf, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    return grid_search.best_estimator_

In [ ]:

# Save predictions to CSV
def save_predictions(predictions, filename):
    test_predictions = pd.DataFrame({
        'text': X_test,
        'processed_text': X_test,
        'predicted_sentiment': predictions
    })
    test_predictions.to_csv(filename, index=False)
    print(f"\nPredictions saved to {filename}")

# Save predictions for both models
save_predictions(linear_predictions, 'test3_linear_svm.csv')
save_predictions(rbf_predictions, 'test3_rbf_svm.csv')